In [71]:
import requests 
import pandas as pd
import time
from pprint import pprint
from ast import literal_eval # an example literal_eval('{"a":5, "b":{"c":1}}')
# from fake_useragent import UserAgent #библиотека для генерации фэйкового юзер-агента
from bs4 import BeautifulSoup #библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных
import socks # для смены ip
import socket # для смены ip

In [72]:
log = pd.DataFrame()

# Заголовки страницы
HEADERS = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
        'Connection': 'keep-alive',
        'Content-Length': '137',
        'content-type': 'application/json',
        'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; navigation_promo_seen-recalls=true',
        'Host': 'auto.ru',
        'origin': 'https://auto.ru',
        'Referer': 'https://auto.ru/moscow/cars/bmw/used/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
        'x-client-app-version': '202002.03.092255',
        'x-client-date': '1590933207763',
        'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
        'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
        'x-requested-with': 'fetch'
    }

marks = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI', 'MERCEDES',
         'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']

years = [['1960', '1980'], ['1980', '2000'], ['2000', '2003'], ['2003', '2007'],
         ['2007', '2010'], ['2010', '2013'], ['2013', '2017'], ['2017', '2021']]

PARAMS = {'catalog_filter': [{'mark': 'MERCEDES'}], 'section': 'used', 
          'category': 'cars', 'sort': 'fresh_relevance_1-desc'}

In [87]:
'''
# this part of code was mede for maximim automation, but in the end i decided to make it more handy 
# to control process, besides it works

for mark in marks:
    PARAMS = [{'catalog_filter': [{'mark': mark}], 'section': 'used', 
               'category': 'cars', 'sort': 'fresh_relevance_1-desc'}]
               '''
# a line below is only for handy part of code
'''mark = 'SKODA'
PARAMS = {'catalog_filter': [{'mark': mark}], 'section': 'used', 
          'category': 'cars', 'sort': 'fresh_relevance_1-desc'}'''

for mark in marks:
    PARAMS = {'catalog_filter': [{'mark': mark}], 'section': 'used', 
               'category': 'cars', 'sort': 'fresh_relevance_1-desc'}
    for year_from, year_to in years:
        for a in range(1, 100):
            PARAMS['year_from'] = year_from
            PARAMS['year_to'] = year_to
            PARAMS['page'] = a
    #        if a == 0 or a == 50 or a == 99:
            if a % 5:
                print(f'mark: {mark}, year from: {year_from}, year_to: {year_to}, page: {a}')
            # URL = 'https://auto.ru/-/ajax/desktop/listing/' #URL на который будет отправлен запрос
            URL = 'https://auto.ru/-/ajax/desktop/listing/?cookiesync=true' #URL на который будет отправлен запрос
            time.sleep(10) # 
            #Параметры запроса

            try:
                response = requests.post(URL, json=PARAMS, headers=HEADERS) #Делаем post запрос на url
                data = response.json()['offers']
                if len(data) == 0:
                    break
            except:
                time.sleep(20)
                print('parser brokes, reload with sleep for 20 secs')
                response = requests.post(URL, json=PARAMS, headers=HEADERS) #Делаем post запрос на url
                data = response.json()['offers']
                print('pass')
                if len(data) == 0:
                    break

            for offer in data:
                car = {}
                try: car['bodyType'] = offer['vehicle_info']['configuration']['human_name']
                except: pass
                try: car['brand'] = offer['vehicle_info']['mark_info']['code']
                except: pass
                try: car['color'] = offer['vehicle_info']['complectation']['vendor_colors'][0]['stock_color']['name_ru']
                except: pass
                try: car['color_hex'] = offer['color_hex'] # v2
                except: pass
                try: car['color2'] = offer['vehicle_info']['complectation']['vendor_colors'][0]['name_ru'] # v2
                except: pass
                try: car['complectation_dict'] = offer['vehicle_info']['complectation']
                except: pass
                try: car['description'] = offer['description']
                except: pass
                try: car['engineDisplacement'] = offer['vehicle_info']['tech_param']['human_name']
                except: pass
                try: car['enginePower'] = offer['vehicle_info']['tech_param']['power']
                except: pass
                try: car['equipment_dict'] = offer['vehicle_info']['equipment']
                except: pass
                try: car['fuelType'] = offer['vehicle_info']['tech_param']['engine_type']
                except: pass
                try: car['mileage'] = offer['state']['mileage']
                except: pass
                try: car['modelDate'] = offer['vehicle_info']['super_gen']['year_from'] # v2
                except: pass
                try: car['model_info'] = offer['vehicle_info']['model_info']
                except: pass
                try: car['model_name'] = offer['vehicle_info']['model_info']['code']
                except: pass
                try: car['name'] = offer['vehicle_info']['tech_param']['human_name']
                except: pass
                try: car['numberOfDoors'] = offer['vehicle_info']['configuration']['doors_count']
                except: pass
                try: car['parsing_unixtime'] = offer['price_history'][-1]['create_timestamp']
                except: pass
                try: car['priceCurrency'] = offer['price_info']['currency']
                except: pass
                try: car['productionDate'] = offer['documents']['year']
                except: pass
                try: car['sell_id'] = offer['id']
                except: pass
                try: car['super_gen'] = offer['vehicle_info']['tech_param']
                except: pass
                try: car['vehicleConfiguration'] = offer['vehicle_info']['configuration']['body_type'] + ' ' + \
                                                   offer['vehicle_info']['tech_param']['transmission'] + ' ' + \
                                                   offer['vehicle_info']['tech_param']['human_name'].split()[0]
                except: pass
                try: car['vehicleTransmission'] = offer['vehicle_info']['tech_param']['transmission']
                except: pass
                try: car['vendor'] = offer['vehicle_info']['vendor']
                except: pass
                try: car['Владельцы'] = offer['documents']['owners_number']
                except: pass
                try: car['Владельцы'] = offer['documents']['owners_number']
                except: pass
                try: car['Владение'] = offer['documents']['purchase_date']
                except: pass
                try: car['ПТС'] = offer['documents']['pts']
                except: pass
                try: car['Привод'] = offer['vehicle_info']['tech_param']['gear_type']
                except: pass
                try: car['Руль'] = offer['vehicle_info']['steering_wheel']
                except: pass
                try: car['Состояние'] = offer['state']['state_not_beaten']
                except: pass
                try: car['Таможня'] = offer['documents']['custom_cleared']
                except: pass
                try: car['Price'] = offer['price_info']['price']
                except: pass

                log = log.append(car,ignore_index=True)
'''    if a % 2:
        log.to_csv(f'{mark}_{year_from}_to_{year_to}_page{a}_1.csv')
    else:
        log.to_csv(f'{mark}_{year_from}_to_{year_to}_page{a}_2.csv')'''

print('Done')

mark: SKODA, year from: 1960, year_to: 1980, page: 1
mark: SKODA, year from: 1960, year_to: 1980, page: 2
mark: SKODA, year from: 1980, year_to: 2000, page: 1
mark: SKODA, year from: 1980, year_to: 2000, page: 2
mark: SKODA, year from: 1980, year_to: 2000, page: 3
mark: SKODA, year from: 2000, year_to: 2003, page: 1
mark: SKODA, year from: 2000, year_to: 2003, page: 2
mark: SKODA, year from: 2000, year_to: 2003, page: 3
mark: SKODA, year from: 2003, year_to: 2007, page: 1
mark: SKODA, year from: 2003, year_to: 2007, page: 2
mark: SKODA, year from: 2003, year_to: 2007, page: 3
mark: SKODA, year from: 2003, year_to: 2007, page: 4
mark: SKODA, year from: 2007, year_to: 2010, page: 1
mark: SKODA, year from: 2007, year_to: 2010, page: 2
mark: SKODA, year from: 2007, year_to: 2010, page: 3
mark: SKODA, year from: 2007, year_to: 2010, page: 4
mark: SKODA, year from: 2007, year_to: 2010, page: 6
mark: SKODA, year from: 2007, year_to: 2010, page: 7
mark: SKODA, year from: 2007, year_to: 2010, p

mark: HONDA, year from: 2000, year_to: 2003, page: 4
mark: HONDA, year from: 2003, year_to: 2007, page: 1
mark: HONDA, year from: 2003, year_to: 2007, page: 2
mark: HONDA, year from: 2003, year_to: 2007, page: 3
mark: HONDA, year from: 2003, year_to: 2007, page: 4
mark: HONDA, year from: 2003, year_to: 2007, page: 6
mark: HONDA, year from: 2007, year_to: 2010, page: 1
mark: HONDA, year from: 2007, year_to: 2010, page: 2
mark: HONDA, year from: 2007, year_to: 2010, page: 3
mark: HONDA, year from: 2007, year_to: 2010, page: 4
mark: HONDA, year from: 2007, year_to: 2010, page: 6
mark: HONDA, year from: 2007, year_to: 2010, page: 7
mark: HONDA, year from: 2007, year_to: 2010, page: 8
mark: HONDA, year from: 2007, year_to: 2010, page: 9
mark: HONDA, year from: 2010, year_to: 2013, page: 1
mark: HONDA, year from: 2010, year_to: 2013, page: 2
mark: HONDA, year from: 2010, year_to: 2013, page: 3
mark: HONDA, year from: 2010, year_to: 2013, page: 4
mark: HONDA, year from: 2010, year_to: 2013, p

mark: BMW, year from: 2013, year_to: 2017, page: 29
mark: BMW, year from: 2013, year_to: 2017, page: 31
mark: BMW, year from: 2013, year_to: 2017, page: 32
mark: BMW, year from: 2013, year_to: 2017, page: 33
mark: BMW, year from: 2013, year_to: 2017, page: 34
mark: BMW, year from: 2013, year_to: 2017, page: 36
mark: BMW, year from: 2013, year_to: 2017, page: 37
mark: BMW, year from: 2013, year_to: 2017, page: 38
mark: BMW, year from: 2013, year_to: 2017, page: 39
mark: BMW, year from: 2017, year_to: 2021, page: 1
mark: BMW, year from: 2017, year_to: 2021, page: 2
mark: BMW, year from: 2017, year_to: 2021, page: 3
mark: BMW, year from: 2017, year_to: 2021, page: 4
mark: BMW, year from: 2017, year_to: 2021, page: 6
mark: BMW, year from: 2017, year_to: 2021, page: 7
mark: BMW, year from: 2017, year_to: 2021, page: 8
mark: BMW, year from: 2017, year_to: 2021, page: 9
mark: BMW, year from: 2017, year_to: 2021, page: 11
mark: BMW, year from: 2017, year_to: 2021, page: 12
mark: BMW, year from

mark: INFINITI, year from: 2013, year_to: 2017, page: 1
mark: INFINITI, year from: 2013, year_to: 2017, page: 2
mark: INFINITI, year from: 2013, year_to: 2017, page: 3
mark: INFINITI, year from: 2013, year_to: 2017, page: 4
mark: INFINITI, year from: 2013, year_to: 2017, page: 6
mark: INFINITI, year from: 2013, year_to: 2017, page: 7
mark: INFINITI, year from: 2013, year_to: 2017, page: 8
mark: INFINITI, year from: 2017, year_to: 2021, page: 1
mark: INFINITI, year from: 2017, year_to: 2021, page: 2
mark: INFINITI, year from: 2017, year_to: 2021, page: 3
mark: INFINITI, year from: 2017, year_to: 2021, page: 4
mark: MERCEDES, year from: 1960, year_to: 1980, page: 1
mark: MERCEDES, year from: 1960, year_to: 1980, page: 2
mark: MERCEDES, year from: 1980, year_to: 2000, page: 1
mark: MERCEDES, year from: 1980, year_to: 2000, page: 2
mark: MERCEDES, year from: 1980, year_to: 2000, page: 3
mark: MERCEDES, year from: 1980, year_to: 2000, page: 4
mark: MERCEDES, year from: 1980, year_to: 2000, 

mark: MERCEDES, year from: 2017, year_to: 2021, page: 27
mark: MERCEDES, year from: 2017, year_to: 2021, page: 28
mark: MERCEDES, year from: 2017, year_to: 2021, page: 29
mark: MERCEDES, year from: 2017, year_to: 2021, page: 31
mark: MERCEDES, year from: 2017, year_to: 2021, page: 32
mark: MERCEDES, year from: 2017, year_to: 2021, page: 33
mark: TOYOTA, year from: 1960, year_to: 1980, page: 1
mark: TOYOTA, year from: 1960, year_to: 1980, page: 2
mark: TOYOTA, year from: 1980, year_to: 2000, page: 1
mark: TOYOTA, year from: 1980, year_to: 2000, page: 2
mark: TOYOTA, year from: 1980, year_to: 2000, page: 3
mark: TOYOTA, year from: 1980, year_to: 2000, page: 4
mark: TOYOTA, year from: 1980, year_to: 2000, page: 6
mark: TOYOTA, year from: 1980, year_to: 2000, page: 7
mark: TOYOTA, year from: 1980, year_to: 2000, page: 8
mark: TOYOTA, year from: 1980, year_to: 2000, page: 9
mark: TOYOTA, year from: 1980, year_to: 2000, page: 11
mark: TOYOTA, year from: 1980, year_to: 2000, page: 12
mark: TO

mark: VOLKSWAGEN, year from: 1980, year_to: 2000, page: 9
mark: VOLKSWAGEN, year from: 1980, year_to: 2000, page: 11
mark: VOLKSWAGEN, year from: 1980, year_to: 2000, page: 12
mark: VOLKSWAGEN, year from: 1980, year_to: 2000, page: 13
mark: VOLKSWAGEN, year from: 2000, year_to: 2003, page: 1
mark: VOLKSWAGEN, year from: 2000, year_to: 2003, page: 2
mark: VOLKSWAGEN, year from: 2000, year_to: 2003, page: 3
mark: VOLKSWAGEN, year from: 2000, year_to: 2003, page: 4
mark: VOLKSWAGEN, year from: 2000, year_to: 2003, page: 6
mark: VOLKSWAGEN, year from: 2000, year_to: 2003, page: 7
mark: VOLKSWAGEN, year from: 2003, year_to: 2007, page: 1
mark: VOLKSWAGEN, year from: 2003, year_to: 2007, page: 2
mark: VOLKSWAGEN, year from: 2003, year_to: 2007, page: 3
mark: VOLKSWAGEN, year from: 2003, year_to: 2007, page: 4
mark: VOLKSWAGEN, year from: 2003, year_to: 2007, page: 6
mark: VOLKSWAGEN, year from: 2003, year_to: 2007, page: 7
mark: VOLKSWAGEN, year from: 2003, year_to: 2007, page: 8
mark: VOLKS

mark: MITSUBISHI, year from: 2007, year_to: 2010, page: 8
mark: MITSUBISHI, year from: 2007, year_to: 2010, page: 9
mark: MITSUBISHI, year from: 2007, year_to: 2010, page: 11
mark: MITSUBISHI, year from: 2007, year_to: 2010, page: 12
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 1
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 2
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 3
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 4
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 6
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 7
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 8
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 9
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 11
mark: MITSUBISHI, year from: 2010, year_to: 2013, page: 12
mark: MITSUBISHI, year from: 2013, year_to: 2017, page: 1
mark: MITSUBISHI, year from: 2013, year_to: 2017, page: 2
mark: MITSUBISHI, year from: 2013, year_to: 2017, page: 3
mark: MITS

In [88]:
log.to_csv('auto_ru_14mar2021_full')

In [89]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35846 entries, 0 to 35845
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Price                 35689 non-null  float64
 1   bodyType              35844 non-null  object 
 2   brand                 35846 non-null  object 
 3   color_hex             35846 non-null  object 
 4   complectation_dict    35846 non-null  object 
 5   description           34759 non-null  object 
 6   engineDisplacement    35844 non-null  object 
 7   enginePower           35844 non-null  float64
 8   equipment_dict        35846 non-null  object 
 9   fuelType              35844 non-null  object 
 10  mileage               35846 non-null  float64
 11  modelDate             35844 non-null  float64
 12  model_info            35846 non-null  object 
 13  model_name            35846 non-null  object 
 14  name                  35844 non-null  object 
 15  numberOfDoors      

piece of code to look manually into one request

In [93]:
URL = 'https://auto.ru/-/ajax/desktop/listing/?cookiesync=true' #URL на который будет отправлен запрос
response = requests.post(URL, json=PARAMS, headers=HEADERS) #Делаем post запрос на url
data = response.json()['offers']

In [100]:
response.json()

{'status': 'SUCCESS',
 'search_parameters': {'year_from': 2017,
  'year_to': 2021,
  'catalog_filter': [{'mark': 'MITSUBISHI'}],
  'section': 'used',
  'category': 'cars'},
 'hiddenOffers': [],
 'offers': [],
 'pagination': {'total_page_count': 5,
  'total_offers_count': 158,
  'page': 1,
  'page_size': 37,
  'from': 1,
  'to': 37,
  'current': 1},
 'price_range': {'min': {'price': 860000,
   'currency': 'RUR',
   'rur_price': 860000,
   'usd_price': 11599,
   'eur_price': 9724,
   'dprice': 860000,
   'rur_dprice': 860000,
   'usd_dprice': 11599,
   'eur_dprice': 9724},
  'max': {'price': 2995000,
   'currency': 'RUR',
   'rur_price': 2995000,
   'usd_price': 40397,
   'eur_price': 33867,
   'dprice': 2995000,
   'rur_dprice': 2995000,
   'usd_dprice': 40397,
   'eur_dprice': 33867}},
 'response_flags': {'show_match_application_form': False},
 'grouping': {},
 'search_id': 'c8402e8d8ae35d9667b5196d85c5f1e73a8adfff',
 'request_id': 'c8dc8fb1b5b93becfa76f2c5f010aefa',
 'query': 'catalog

In [ ]:
# 2.0 LTR 

In [46]:
data[0]['vehicle_info'].keys()

dict_keys(['configuration', 'complectation', 'equipment', 'mark_info', 'model_info', 'steering_wheel', 'super_gen', 'tech_param', 'vendor'])

In [12]:
data[0].keys()

dict_keys(['availability', 'category', 'color_hex', 'delivery_info', 'description', 'discount_options', 'documents', 'is_favorite', 'old_category_id', 'owner_expenses', 'score', 'section', 'seller_type', 'services', 'status', 'tags', 'additional_info', 'counters', 'grouping_info', 'hash', 'id', 'price_info', 'price_history', 'recall_info', 'relevance', 'saleId', 'salon', 'seller', 'service_prices', 'state', 'sub_category', 'vehicle_info', 'lk_summary', 'multiposting'])

In [70]:
data[0]['vehicle_info']['tech_param']

{'id': '22347487',
 'name': '63 AMG S',
 'nameplate': '63 AMG S',
 'displacement': 3982,
 'engine_type': 'GASOLINE',
 'gear_type': 'ALL_WHEEL_DRIVE',
 'transmission': 'AUTOMATIC',
 'power': 612,
 'power_kvt': 450,
 'human_name': '63 AMG S 4.0 AT (612 л.с.) 4WD',
 'acceleration': 3.4,
 'clearance_min': 114,
 'fuel_rate': 11.6}

In [97]:
data

[]

In [ ]:
data[0]['vehicle_info']['complectation'] - еще тут есть название цвета

In [78]:
for a in range(1, 3):
    print(a.__class__)

<class 'int'>
<class 'int'>
